<a href="https://colab.research.google.com/github/deitrashafira/LSTM-Forecasting-StockPrices/blob/main/LSTM-Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building LSTM Models For Forecasting PJAA's Stock Prices

This research uses Google Colaboratory with Python programming language to build LSTM model in order to forecast the stock prices of PT Pembangunan Jaya Ancol Tbk (PJAA). First thing first, we have to connect the Google Drive to Google Colaboratory with the aim of importing dataset easily.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


There are several libraries that need to be imported and below are those.

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from tensorflow import keras
from keras import backend as K
from keras import Model
from keras import callbacks
from keras.layers import LSTM, Input, Dense
from keras.models import Sequential

Then, we import the stored dataset in Google Drive and define the used variables. In addition, we create the table containing all those research variables, but the date in `data_wo_date`.

In [ ]:
df = pd.read_excel('/content/PJAA.JK 31 Oktober.xlsx')
Close = df['Close']
Open = df['Open']
Low = df['Low']
High = df['High']
data_wo_date = pd.DataFrame({'Close': df['Close'],
                   'Open': df['Open'],
                   'Low': df['Low'],
                   'High': df['High']})

To know the descriptive statistics in PJAA's stock prices, as Kuncoro (2020) stated, candlestick plot would be great to use for tracking the stock prices volatility. Below are its code and result.

In [ ]:
# Data exploration using candlestick
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])
fig.update_layout(
    title='Candlestick Plot Harga Saham PJAA',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

From candlestick plot above, what we can derive is the nonstationary pattern of prices from July 2005 to October 2022. Throughout those years, PJAA hit the highest price at 2,875 rupiahs for each share on January 2015. It happened since the company needed to develop projects until 2020. In contrast, the lowest price that PJAA ever reached was at 320 rupiahs for each share. At that time, on December 2008 to be exact, the world has underwent the economic global crisis. As such, it declined to 320 rupiahs.

More functions to build will ease us to construct LSTM model ahead. Here are some functions for the future use.

`transformasi_data` function is used to rescale the data into a range of [0,1] using Min-Max Scaler. This will lead to fasten the model convergence to meet the optimal weights and biases as well as avoid the domination of certain values in dataset.

In [ ]:
def transformasi_data(x):
  x_trans = (x-min(df['Low']))/(max(df['High'])-min(df['Low']))
  return x_trans

`transformasi_data` function will be used for the input variables, whereas the `transformasi_balik` function is built to rescale the transformed data into the former unit - for this case it will be scaled from [0,1] to rupiahs.

In [ ]:
def transformasi_balik(x):
  x_trans_balik = x * (max(df['High'])-min(df['Low'])) + min(df['Low'])
  return x_trans_balik

The input variables depends on the lag of data which is checked from PCCF in SAS code in this repository. PCCF utilize the several lags from each variable only if its lag is significant. The `unit_input` function turns the univariate variable into the required lags.

In [ ]:
# penentuan unit input, tinggal cut bagian atas yang bernilai 0
def unit_input(x, lag):
  n = len(x)
  list_data_lag = [0] * n
  list_data_lag[lag:n] = x[0:(n-lag)]
  return list_data_lag

This research uses the Masters theory in 1993. He conveyed that there is no rules to define the number of hidden layers in a model architecture, but the hidden neurons. The number of its neuron should be in between the input and output variables. For example, if we have 4 output variables and 8 input variables, then there are 5, 6, and 7 hidden neurons in maximal for each layer in a model. Below are the functions to build the LSTM model containing 1, 2, and 3 hidden layers.

In [ ]:
def model_1_layer(units):
  model = keras.models.Sequential()
  model.add(keras.layers.LSTM(units = units, input_shape = (x_train_reshape.shape[1], x_train_reshape.shape[2]),
                            recurrent_activation="sigmoid", activation="tanh"))
  model.add(keras.layers.Dense(4, activation = "sigmoid"))
  model.compile(optimizer = 'adam',
              loss = root_mean_squared_error)
  earlystopping = callbacks.EarlyStopping(monitor = "val_loss",
                                        mode = "min",
                                        patience = 15,
                                        restore_best_weights=True)
  model.fit(x_train_reshape, y_train, epochs=300, batch_size=30,
          callbacks=earlystopping, validation_data=(x_test_reshape, y_test))
  return model

In [ ]:
def model_2_layer(unit1, unit2):
  model = keras.models.Sequential()
  model.add(keras.layers.LSTM(units = unit1, input_shape = (x_train_reshape.shape[1], x_train_reshape.shape[2]),
                            recurrent_activation="sigmoid", activation="tanh", return_sequences=True))
  model.add(keras.layers.LSTM(units = unit2, recurrent_activation="sigmoid", activation="tanh"))
  model.add(keras.layers.Dense(4, activation = "sigmoid"))
  model.compile(optimizer = 'adam',
              loss = root_mean_squared_error)
  earlystopping = callbacks.EarlyStopping(monitor = "val_loss",
                                        mode = "min",
                                        patience = 15,
                                        restore_best_weights=True)
  model.fit(x_train_reshape, y_train, epochs=300, batch_size=30,
          callbacks=earlystopping, validation_data=(x_test_reshape, y_test))
  return model

In [ ]:
def model_3_layer(unit1, unit2, unit3):
  model = keras.models.Sequential()
  model.add(keras.layers.LSTM(units = unit1, input_shape = (x_train_reshape.shape[1], x_train_reshape.shape[2]),
                            recurrent_activation="sigmoid", activation="tanh", return_sequences=True))
  model.add(keras.layers.LSTM(units = unit2, return_sequences=True, recurrent_activation="sigmoid", activation="tanh"))
  model.add(keras.layers.LSTM(units = unit3, recurrent_activation="sigmoid", activation="tanh"))
  model.add(keras.layers.Dense(4, activation = "sigmoid"))
  model.compile(optimizer = 'adam',
              loss = root_mean_squared_error)
  earlystopping = callbacks.EarlyStopping(monitor = "val_loss",
                                        mode = "min",
                                        patience = 15,
                                        restore_best_weights=True)
  model.fit(x_train_reshape, y_train, epochs=300, batch_size=30,
          callbacks=earlystopping, validation_data=(x_test_reshape, y_test))
  return model

After the best model is chosen, then we need to forecast the stock prices in a serveral periods ahead. The `forecast` function will do its role to make the earlier prices as the input to project the prices in the next day.

In [ ]:
def forecast(y, model):
  list_prediction = []
  list_data = [y[-6], y[-5], y[-4], y[-3], y[-2], y[-1]]
  for i in range(len(y_test)):
    list_input = []
    for j in [-1, -2, -3, -4, -5, -6]:
      for k in list_data[j]:
        list_input.append(k)
    for i in [1,1,3,3,5,5,5,6,6,6,7,7,7,8,8,8]:
      list_input.remove(list_input[i])
    list_input = np.asarray(list_input)
    input = list_input.reshape(1, 1, 8)
    predict = model.predict(input)
    for l in predict:
      list_data.append(l.tolist())
      list_prediction.append(l.tolist())
  return np.asarray(list_prediction)

Before choosing the best model, there actually are some LSTM built models. We will choose the best model by its value in RMSE using `rmse` function and MAPE using `mape` function to compare the actual test data and its forecast.

In [ ]:
def rmse(y_pred, y_act):
  n = len(y_pred)
  selisih_kuadrat = 0
  for i in range(0, n):
    selisih_kuadrat += (y_act[i] - y_pred[i]) **2
  rmse = (selisih_kuadrat / n) ** (1/2)
  return rmse

In [ ]:
def mape(y_pred, y_act):
  n = len(y_pred)
  selisih_absolut = 0
  for i in range(0, n):
    selisih_absolut += abs((y_act[i] - y_pred[i]) / y_act[i])
  mape = (selisih_absolut * 100) / n
  return mape

Probably the forecasting prices may result in the wrong relationship amongst 4 output variables. That means, the forecasted lowest price may be higher than the highest price and vice versa. The `high_low_check` will be on duty to verify whether the forecasting results in the fulfilling condition or not. If the function returns the 100% value, then the desired conditions are satisfied already by the forecasting result.

In [ ]:
def high_low_check(df):
  count = 0
  for i in range(len(df)):
    if (df['Close'][i] > df['Low'][i]) or (df['Open'][i] > df['Low'][i]) or (df['High'][i] > df['Low'][i]) or (df['Close'][i] < df['High'][i]) or (df['Open'][i] < df['High'][i]):
      count += 1
  return("{}%".format((count/len(df))*100))

The `gabung_variabel` function will be used to remake a few columns in a row.

In [ ]:
# fungsi gabung jadi satu array untuk perhitungan rmse dan mape
def gabung_variabel(a, b, c, d):
  list_variabel = []
  for i in a.tolist():
    list_variabel.append(i)
  for i in b.tolist():
    list_variabel.append(i)
  for i in c.tolist():
    list_variabel.append(i)
  for i in d.tolist():
    list_variabel.append(i)
  return list_variabel

The last function is `root_mean_squared_error` that will be placed in training the model to inspect the RMSE of training loss and validation loss.

In [ ]:
# rmse untuk error pada pelatihan model
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

Here we copy the `data_wo_date` for the future use. Then, the actual data that contains 4 variables will be transformed using `transformasi_data` function. It's stored in the `data_copy`.

In [ ]:
data_copy = data_wo_date.copy()

In [ ]:
for i in data_copy.columns:
  data_copy[i] = transformasi_data(data_copy[i])

In [ ]:
data_copy

,Close,Open,Low,High
0,0.075342,0.075342,0.075342,0.075342
1,0.075342,0.080235,0.070450,0.085127
2,0.070450,0.065558,0.065558,0.070450
3,0.075342,0.070450,0.065558,0.075342
4,0.075342,0.070450,0.070450,0.075342
...,...,...,...,...
4512,0.168297,0.158513,0.156556,0.168297
4513,0.158513,0.162427,0.158513,0.164384
4514,0.158513,0.158513,0.148728,0.158513
4515,0.152642,0.154599,0.148728,0.158513


Based on the R code, things we derived is the Augmented Dickey Fuller test results in rejecting null hypothesis after data differencing on first lag. Therefore, all of variables are mean stationary after the first differencing.

In addition, from the SAS code using PCCF, we know that all variables in each lag are significant. It is called significant when the partial correlation among 2 variables in particular lag shows the (+) or (-) symbol. It will results the (+) symbol if the correlation value is greater than `2/sqrt(n)`, while (-) symbol if the value is smaller than `-2/sqrt(n)`. However, if all the significant variables are included as the model input, the model performance will decrease as there are too many inputs. Hence, we calculate the new cut-off value to diminish the number of inputs, which is `4/sqrt(n)`. The partial correlation value will be given (+) symbol if it's greater than `4/sqrt(n)`, (-) symbol if it's lower than `-4/sqrt(n)`, while (.) symbol if it's in between `-4/sqrt(n)` and `4/sqrt(n)`.

PCCF analysis which is aligned with the new cut-off value gave us the information on lags that will be used. The significant partial correlation to 4 variables are Close at first to sixth lags and High at first and two lags. As such, we define each variables on its lags below.


In [ ]:
Close_t1 = unit_input(data_copy['Close'], 1)
Close_t2 = unit_input(data_copy['Close'], 2)
Close_t3 = unit_input(data_copy['Close'], 3)
Close_t4 = unit_input(data_copy['Close'], 4)
Close_t5 = unit_input(data_copy['Close'], 5)
Close_t6 = unit_input(data_copy['Close'], 6)
High_t1 = unit_input(data_copy['High'], 1)
High_t2 = unit_input(data_copy['High'], 2)

"\nLow_t1 = unit_input(data_copy['Low'], 1)\nLow_t2 = unit_input(data_copy['Low'], 2)\n"

Then, all of variables suited with its lags are combined into a table named `data`.

In [ ]:
data = pd.DataFrame({'Close(t)': data_copy['Close'],
                     'Open(t)': data_copy['Open'],
                     'Low(t)': data_copy['Low'],
                     'High(t)': data_copy['High'],
                     'Close(t-1)': Close_t1,
                     'High(t-1)': High_t1,
                     'Close(t-2)': Close_t2,
                     'High(t-2)': High_t2,
                     'Close(t-3)': Close_t3,
                     'Close(t-4)': Close_t4,
                     'Close(t-5)': Close_t5,
                     'Close(t-6)': Close_t6})

In [ ]:
data

,Close(t),Open(t),Low(t),High(t),Close(t-1),High(t-1),Close(t-2),High(t-2),Close(t-3),Close(t-4),Close(t-5),Close(t-6)
0,0.075342,0.075342,0.075342,0.075342,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.075342,0.080235,0.070450,0.085127,0.075342,0.075342,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.070450,0.065558,0.065558,0.070450,0.075342,0.085127,0.075342,0.075342,0.000000,0.000000,0.000000,0.000000
3,0.075342,0.070450,0.065558,0.075342,0.070450,0.070450,0.075342,0.085127,0.075342,0.000000,0.000000,0.000000
4,0.075342,0.070450,0.070450,0.075342,0.075342,0.075342,0.070450,0.070450,0.075342,0.075342,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
4512,0.168297,0.158513,0.156556,0.168297,0.160470,0.168297,0.172211,0.172211,0.152642,0.138943,0.152642,0.156556
4513,0.158513,0.162427,0.158513,0.164384,0.168297,0.168297,0.160470,0.168297,0.172211,0.152642,0.138943,0.152642
4514,0.158513,0.158513,0.148728,0.158513,0.158513,0.164384,0.168297,0.168297,0.160470,0.172211,0.152642,0.138943
4515,0.152642,0.154599,0.148728,0.158513,0.158513,0.158513,0.158513,0.164384,0.168297,0.160470,0.172211,0.152642


The maximum lag we got is 6, thus to be able to use data in time series analysis, we do remove the top 6 rows observation.

In [ ]:
data = data.drop([0,1,2,3,4,5], axis=0)

Then, we reset the indexing of the dataset table, so the period index will start at 0.

In [ ]:
data = data.reset_index(drop=True)

And here is the dataset table, namely `data` which contains the 8 input variables (`Close(t-1)`, ..., `Close(t-6)`) and 4 output variables (`Close(t)`, ..., `High(t)`).

In [ ]:
data

,Close(t),Open(t),Low(t),High(t),Close(t-1),High(t-1),Close(t-2),High(t-2),Close(t-3),Close(t-4),Close(t-5),Close(t-6)
0,0.075342,0.075342,0.070450,0.075342,0.075342,0.075342,0.075342,0.075342,0.075342,0.070450,0.075342,0.075342
1,0.070450,0.070450,0.065558,0.070450,0.075342,0.075342,0.075342,0.075342,0.075342,0.075342,0.070450,0.075342
2,0.070450,0.065558,0.065558,0.070450,0.070450,0.070450,0.075342,0.075342,0.075342,0.075342,0.075342,0.070450
3,0.070450,0.065558,0.065558,0.070450,0.070450,0.070450,0.070450,0.070450,0.075342,0.075342,0.075342,0.075342
4,0.070450,0.065558,0.065558,0.070450,0.070450,0.070450,0.070450,0.070450,0.070450,0.075342,0.075342,0.075342
...,...,...,...,...,...,...,...,...,...,...,...,...
4506,0.168297,0.158513,0.156556,0.168297,0.160470,0.168297,0.172211,0.172211,0.152642,0.138943,0.152642,0.156556
4507,0.158513,0.162427,0.158513,0.164384,0.168297,0.168297,0.160470,0.168297,0.172211,0.152642,0.138943,0.152642
4508,0.158513,0.158513,0.148728,0.158513,0.158513,0.164384,0.168297,0.168297,0.160470,0.172211,0.152642,0.138943
4509,0.152642,0.154599,0.148728,0.158513,0.158513,0.158513,0.158513,0.164384,0.168297,0.160470,0.172211,0.152642


To prepare the dataset in model training, we do need splitting dataset into training and testing data. I decided to set the proportion on 95% for the training data, while the rest for testing data. It's caused by the number of periods ahead we will forecast in stock prices.

In [ ]:
split_percent = 0.95
split_data = int(split_percent * len(data_copy))

Stock prices from July 12, 2005 to December 6, 2021 will be a training set, while prices from December 7, 2021 to October 31, 2022 will be merged as a testing set.

In [ ]:
# data train dan test sebelum transformasi
data_drop_4rows = data_wo_date.drop([0,1,2,3,4,5], axis=0)
data_drop_4rows.reset_index(drop=True)

,Close,Open,Low,High
0,512.5,512.5,500.0,512.5
1,500.0,500.0,487.5,500.0
2,500.0,487.5,487.5,500.0
3,500.0,487.5,487.5,500.0
4,500.0,487.5,487.5,500.0
...,...,...,...,...
4506,750.0,725.0,720.0,750.0
4507,725.0,735.0,725.0,740.0
4508,725.0,725.0,700.0,725.0
4509,710.0,715.0,700.0,725.0


In [ ]:
y_test_asli = data_drop_4rows.iloc[split_data:,]
y_test_asli

,Close,Open,Low,High
4297,560.0,555.0,555.0,560.0
4298,570.0,565.0,560.0,575.0
4299,575.0,570.0,570.0,580.0
4300,565.0,575.0,555.0,575.0
4301,585.0,565.0,565.0,585.0
...,...,...,...,...
4512,750.0,725.0,720.0,750.0
4513,725.0,735.0,725.0,740.0
4514,725.0,725.0,700.0,725.0
4515,710.0,715.0,700.0,725.0


In [ ]:
y_train_asli = data_drop_4rows.iloc[:split_data,]
y_train_asli

,Close,Open,Low,High
6,512.5,512.5,500.0,512.5
7,500.0,500.0,487.5,500.0
8,500.0,487.5,487.5,500.0
9,500.0,487.5,487.5,500.0
10,500.0,487.5,487.5,500.0
...,...,...,...,...
4292,550.0,565.0,545.0,570.0
4293,545.0,545.0,540.0,545.0
4294,555.0,555.0,545.0,555.0
4295,565.0,555.0,555.0,570.0


In [ ]:
# data train dan test setelah transformasi
x_train, y_train = data.iloc[:split_data, 4:], data.iloc[:split_data, 0:4]
x_test, y_test = data.iloc[split_data:, 4:], data.iloc[split_data:, 0:4]
x_train, y_train, x_test, y_test = np.asarray(x_train), np.asarray(y_train), np.asarray(x_test), np.asarray(y_test)

In [ ]:
x_train_reshape = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
x_test_reshape = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))

In [ ]:
print(x_train_reshape.shape)
print(x_test_reshape.shape)

(4291, 1, 8)
(220, 1, 8)


Based on what Masters (1993) delivered, in this research we will build 6 LSTM models with details below.
1. `model1` with 1 hidden layer, containing 5 hidden neurons in it.
2. `model2` with 1 hidden layer, containing 6 hidden neurons in it.
3. `model3` with 2 hidden layers, containing 6 and 5 hidden neurons respectively.
4. `model4` with 2 hidden layers, containing 7 and 6 hidden neurons respectively.
5. `model5` with 3 hidden layers, containing 6, 5, and 4 hidden neurons respectively.
6. `model6` with 3 hidden layers, containing 7, 6, and 5 hidden neurons respectively.


This is the core of our research, training and building the LSTM model to find the best model for forecasting PJAA's stock prices. Here are the process of training the LSTM models, using Adam as an optimizer technique and RMSE as the loss function.

In [ ]:
model1 = model_1_layer(5)

Epoch 1/300
144/144 [==============================] - 2s 6ms/step - loss: 0.2762 - val_loss: 0.3412
Epoch 2/300
144/144 [==============================] - 1s 4ms/step - loss: 0.2299 - val_loss: 0.2531
Epoch 3/300
144/144 [==============================] - 1s 4ms/step - loss: 0.1535 - val_loss: 0.1429
Epoch 4/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0866 - val_loss: 0.0677
Epoch 5/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0552 - val_loss: 0.0373
Epoch 6/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0455 - val_loss: 0.0265
Epoch 7/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0401 - val_loss: 0.0196
Epoch 8/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0359 - val_loss: 0.0151
Epoch 9/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0324 - val_loss: 0.0124
Epoch 10/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0294 - val_lo

In [ ]:
model2 = model_1_layer(6)

Epoch 1/300
144/144 [==============================] - 2s 6ms/step - loss: 0.2746 - val_loss: 0.3332
Epoch 2/300
144/144 [==============================] - 0s 3ms/step - loss: 0.2236 - val_loss: 0.2352
Epoch 3/300
144/144 [==============================] - 1s 4ms/step - loss: 0.1560 - val_loss: 0.1366
Epoch 4/300
144/144 [==============================] - 1s 4ms/step - loss: 0.1032 - val_loss: 0.0718
Epoch 5/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0731 - val_loss: 0.0405
Epoch 6/300
144/144 [==============================] - 1s 8ms/step - loss: 0.0585 - val_loss: 0.0280
Epoch 7/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0513 - val_loss: 0.0211
Epoch 8/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0473 - val_loss: 0.0178
Epoch 9/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0443 - val_loss: 0.0166
Epoch 10/300
144/144 [==============================] - 1s 4ms/step - loss: 0.0422 - val_lo

In [ ]:
model3 = model_2_layer(6,5)

Epoch 1/300
144/144 [==============================] - 4s 9ms/step - loss: 0.2722 - val_loss: 0.3222
Epoch 2/300
144/144 [==============================] - 1s 5ms/step - loss: 0.1929 - val_loss: 0.1434
Epoch 3/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0828 - val_loss: 0.0394
Epoch 4/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0570 - val_loss: 0.0307
Epoch 5/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0491 - val_loss: 0.0243
Epoch 6/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0395 - val_loss: 0.0164
Epoch 7/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0304 - val_loss: 0.0112
Epoch 8/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0248 - val_loss: 0.0087
Epoch 9/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0223 - val_loss: 0.0076
Epoch 10/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0211 - val_lo

In [ ]:
model4 = model_2_layer(7,6)

Epoch 1/300
144/144 [==============================] - 5s 9ms/step - loss: 0.2704 - val_loss: 0.3114
Epoch 2/300
144/144 [==============================] - 1s 5ms/step - loss: 0.1822 - val_loss: 0.1333
Epoch 3/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0812 - val_loss: 0.0448
Epoch 4/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0577 - val_loss: 0.0320
Epoch 5/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0489 - val_loss: 0.0251
Epoch 6/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0412 - val_loss: 0.0195
Epoch 7/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0343 - val_loss: 0.0149
Epoch 8/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0296 - val_loss: 0.0122
Epoch 9/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0256 - val_loss: 0.0106
Epoch 10/300
144/144 [==============================] - 1s 5ms/step - loss: 0.0231 - val_lo

In [ ]:
model5 = model_3_layer(6,5,4)

Epoch 1/300
144/144 [==============================] - 6s 12ms/step - loss: 0.2714 - val_loss: 0.3141
Epoch 2/300
144/144 [==============================] - 1s 6ms/step - loss: 0.2050 - val_loss: 0.1623
Epoch 3/300
144/144 [==============================] - 1s 6ms/step - loss: 0.1186 - val_loss: 0.0465
Epoch 4/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0741 - val_loss: 0.0308
Epoch 5/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0647 - val_loss: 0.0303
Epoch 6/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0585 - val_loss: 0.0275
Epoch 7/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0537 - val_loss: 0.0255
Epoch 8/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0493 - val_loss: 0.0224
Epoch 9/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0451 - val_loss: 0.0194
Epoch 10/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0408 - val_l

In [ ]:
model6 = model_3_layer(7,6,5)

Epoch 1/300
144/144 [==============================] - 6s 13ms/step - loss: 0.2679 - val_loss: 0.2924
Epoch 2/300
144/144 [==============================] - 1s 6ms/step - loss: 0.1721 - val_loss: 0.0862
Epoch 3/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0798 - val_loss: 0.0227
Epoch 4/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0618 - val_loss: 0.0243
Epoch 5/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0544 - val_loss: 0.0245
Epoch 6/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0480 - val_loss: 0.0233
Epoch 7/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0416 - val_loss: 0.0194
Epoch 8/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0347 - val_loss: 0.0155
Epoch 9/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0286 - val_loss: 0.0112
Epoch 10/300
144/144 [==============================] - 1s 6ms/step - loss: 0.0243 - val_l

After 6 model architectures were successfully built, then we forecast the testing set using each model. The `forecast` function we made earlier is on its job. `y_hat_test1` is the result of testing set forecasting from `model1`, and it goes until `y_hat_test6` which is the result of testing set forecasting from `model6`.

In [ ]:
y_hat_test1 = forecast(y_train, model1)
y_hat_test2 = forecast(y_train, model2)
y_hat_test3 = forecast(y_train, model3)
y_hat_test4 = forecast(y_train, model4)
y_hat_test5 = forecast(y_train, model5)
y_hat_test6 = forecast(y_train, model6)

1/1 [==============================] - 0s 18ms/step


The prices forecasting results are yet in form of [0,1] from Min-Max transformation. Thus, we need to rescale the value of forecasting to the rupiahs unit in order to be well-interpreted.

In [ ]:
# mentransformasi balik hasil prediksi data testing tiap model
y_hat_close1 = transformasi_balik(y_hat_test1[:,0])
y_hat_open1 = transformasi_balik(y_hat_test1[:,1])
y_hat_low1 = transformasi_balik(y_hat_test1[:,2])
y_hat_high1 = transformasi_balik(y_hat_test1[:,3])

y_hat_close2 = transformasi_balik(y_hat_test2[:,0])
y_hat_open2 = transformasi_balik(y_hat_test2[:,1])
y_hat_low2 = transformasi_balik(y_hat_test2[:,2])
y_hat_high2 = transformasi_balik(y_hat_test2[:,3])

y_hat_close3 = transformasi_balik(y_hat_test3[:,0])
y_hat_open3 = transformasi_balik(y_hat_test3[:,1])
y_hat_low3 = transformasi_balik(y_hat_test3[:,2])
y_hat_high3 = transformasi_balik(y_hat_test3[:,3])

y_hat_close4 = transformasi_balik(y_hat_test4[:,0])
y_hat_open4 = transformasi_balik(y_hat_test4[:,1])
y_hat_low4 = transformasi_balik(y_hat_test4[:,2])
y_hat_high4 = transformasi_balik(y_hat_test4[:,3])

y_hat_close5 = transformasi_balik(y_hat_test5[:,0])
y_hat_open5 = transformasi_balik(y_hat_test5[:,1])
y_hat_low5 = transformasi_balik(y_hat_test5[:,2])
y_hat_high5 = transformasi_balik(y_hat_test5[:,3])

y_hat_close6 = transformasi_balik(y_hat_test6[:,0])
y_hat_open6 = transformasi_balik(y_hat_test6[:,1])
y_hat_low6 = transformasi_balik(y_hat_test6[:,2])
y_hat_high6 = transformasi_balik(y_hat_test6[:,3])

After that, we make forecasting table from each LSTM model. The result of forecasting as we expected, includes 4 variables, which are Close, Open, Low, and High for 220 periods.


`tabel_pred1` is a table contains forecasting result from `model1` that is already rescaled, and it goes the same as `tabel_pred6`, which a forecasting result table from `model6`.

In [ ]:
tabel_pred1 = pd.DataFrame({'Close': y_hat_close1, 'Open': y_hat_open1, 'Low': y_hat_low1, 'High': y_hat_high1})
tabel_pred2 = pd.DataFrame({'Close': y_hat_close2, 'Open': y_hat_open2, 'Low': y_hat_low2, 'High': y_hat_high2})
tabel_pred3 = pd.DataFrame({'Close': y_hat_close3, 'Open': y_hat_open3, 'Low': y_hat_low3, 'High': y_hat_high3})
tabel_pred4 = pd.DataFrame({'Close': y_hat_close4, 'Open': y_hat_open4, 'Low': y_hat_low4, 'High': y_hat_high4})
tabel_pred5 = pd.DataFrame({'Close': y_hat_close5, 'Open': y_hat_open5, 'Low': y_hat_low5, 'High': y_hat_high5})
tabel_pred6 = pd.DataFrame({'Close': y_hat_close6, 'Open': y_hat_open6, 'Low': y_hat_low6, 'High': y_hat_high6})

We want to check whether all the forecasting result shows the correct relation among 4 variables. The right conditions are as follows.
1. The lowest price is always lower than or equals to the closing, opening, and highest price.
2. The highest price is always higher than or equals to the closing, opening, and lowest price.
3. The closing and opening price are always in between the lowest and highest price.

`high_low_check` function will turn the correct condition for each forecasting result from the models in a percentage unit. 

In [ ]:
high_low_test = [['Model 1', high_low_check(tabel_pred1)], ['Model 2', high_low_check(tabel_pred2)], ['Model 3', high_low_check(tabel_pred3)],
                 ['Model 4', high_low_check(tabel_pred4)], ['Model 5', high_low_check(tabel_pred5)], ['Model 6', high_low_check(tabel_pred6)]]

In [ ]:
tabel_high_low_test = pd.DataFrame(high_low_test, columns=['Model','Persentase Ketepatan Ramalan Data Uji'])
tabel_high_low_test

,Model,Persentase Ketepatan Ramalan Data Uji
0,Model 1,100.0%
1,Model 2,100.0%
2,Model 3,100.0%
3,Model 4,100.0%
4,Model 5,100.0%
5,Model 6,100.0%


From table above, we can conclude that 6 LSTM models result in the correct relation in forecasting testing set prices, they're shown by the 100 in percentage. To pick one out of six models to forecast, we should weigh the RMSE which is derived from the testing set forecasting and the actual testing set.

In [ ]:
y_hat1 = gabung_variabel(y_hat_close1, y_hat_open1, y_hat_low1, y_hat_high1)
y_hat2 = gabung_variabel(y_hat_close2, y_hat_open2, y_hat_low2, y_hat_high2)
y_hat3 = gabung_variabel(y_hat_close3, y_hat_open3, y_hat_low3, y_hat_high3)
y_hat4 = gabung_variabel(y_hat_close4, y_hat_open4, y_hat_low4, y_hat_high4)
y_hat5 = gabung_variabel(y_hat_close5, y_hat_open5, y_hat_low5, y_hat_high5)
y_hat6 = gabung_variabel(y_hat_close6, y_hat_open6, y_hat_low6, y_hat_high6)

y = gabung_variabel(y_test_asli.iloc[:,0], y_test_asli.iloc[:,1], y_test_asli.iloc[:,2], y_test_asli.iloc[:,3])

In [ ]:
# membuat tabel rmse dan mape

evaluasi_model = [['Model 1', rmse(y_hat1, y), mape(y_hat1, y)], ['Model 2', rmse(y_hat2, y), mape(y_hat2, y)], ['Model 3', rmse(y_hat3, y), mape(y_hat3, y)],
                  ['Model 4', rmse(y_hat4, y), mape(y_hat4, y)], ['Model 5', rmse(y_hat5, y), mape(y_hat5, y)], ['Model 6', rmse(y_hat6, y), mape(y_hat6, y)]]

tabel_evaluasi = pd.DataFrame(evaluasi_model, columns=['Model','RMSE','MAPE'])

In [ ]:
tabel_evaluasi = pd.DataFrame(evaluasi_model, columns=['Model','RMSE','MAPE'])
tabel_evaluasi

,Model,RMSE,MAPE
0,Model 1,114.310764,13.233900
1,Model 2,110.517402,12.645289
2,Model 3,63.328748,9.799828
3,Model 4,148.247130,17.786069
4,Model 5,99.769525,11.238395
5,Model 6,139.033567,16.389155


`tabel_evaluasi` gives us the conclusion on the best model we should choose between those 6 LSTM models. Model 3 which contains 2 hidden layers, where 6 and 5 neurons respectively inside those layers, is the best model. Model 3 has the lowest metrics in forecasting accuracy, 63.33 on RMSE and 9.79 in MAPE. 

In order to give us the visualization on how good the Model 3 forecast the testing set of PJAA's stock prices, here we plot the actual and forecasting result to see the differences between them. Each plot shows the difference of its actual and forecast value for each variables as well: Close, Open, Low, High. 

In [ ]:
# Perbandingan data aktual dan ramalan data uji Close

trace2 = go.Scatter(x=df['Date'][4297:],
                    y=y_test_asli['Close'],
                    mode='lines',
                    name='Data Uji')
trace3 = go.Scatter(x=df['Date'][4297:],
                    y=y_hat_close3,
                    mode='lines',
                    name='Ramalan Data Uji')
fig = go.Figure(data=[trace2, trace3])
fig.update_layout(
    title='Plot Perbandingan Data Aktual dan Ramalan pada Data Uji',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

In [ ]:
# Perbandingan data aktual dan ramalan data uji Open

trace2 = go.Scatter(x=df['Date'][4297:],
                    y=y_test_asli['Open'],
                    mode='lines',
                    name='Data Uji')
trace3 = go.Scatter(x=df['Date'][4297:],
                    y=y_hat_open3,
                    mode='lines',
                    name='Ramalan Data Uji')
fig = go.Figure(data=[trace2, trace3])
fig.update_layout(
    title='Plot Perbandingan Data Aktual dan Ramalan pada Data Uji',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

In [ ]:
# Perbandingan data aktual dan ramalan data uji Open

trace2 = go.Scatter(x=df['Date'][4297:],
                    y=y_test_asli['Low'],
                    mode='lines',
                    name='Data Uji')
trace3 = go.Scatter(x=df['Date'][4297:],
                    y=y_hat_low3,
                    mode='lines',
                    name='Ramalan Data Uji')
fig = go.Figure(data=[trace2, trace3])
fig.update_layout(
    title='Plot Perbandingan Data Aktual dan Ramalan pada Data Uji',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

In [ ]:
# Perbandingan data aktual dan ramalan data uji Open

trace2 = go.Scatter(x=df['Date'][4297:],
                    y=y_test_asli['High'],
                    mode='lines',
                    name='Data Uji')
trace3 = go.Scatter(x=df['Date'][4297:],
                    y=y_hat_high3,
                    mode='lines',
                    name='Ramalan Data Uji')
fig = go.Figure(data=[trace2, trace3])
fig.update_layout(
    title='Plot Perbandingan Data Aktual dan Ramalan pada Data Uji',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

We found the Model 3 as the best model. Then, we do forecast for PJAA's stock prices from November 1, 2022 to September 6, 2023 as the process is provided below.

In [ ]:
# model terbaik: model 3
y_ramal = forecast(y_test, model3)
y_close = transformasi_balik(y_ramal[:,0])
y_open = transformasi_balik(y_ramal[:,1])
y_low = transformasi_balik(y_ramal[:,2])
y_high = transformasi_balik(y_ramal[:,3])
tabel_ramal = pd.DataFrame({'Close': y_close, 'Open': y_open, 'Low': y_low, 'High': y_high})

1/1 [==============================] - 0s 24ms/step


Below is the result of stock price forecasting for 220 periods ahead.

In [ ]:
tabel_ramal

,Close,Open,Low,High
0,712.702343,710.328943,698.841300,721.540295
1,712.206258,709.802667,698.283690,721.070062
2,713.171890,710.765254,699.196440,722.095734
3,713.071075,710.663410,699.085382,722.016734
4,713.630055,711.211653,699.608460,722.582072
...,...,...,...,...
215,744.548705,741.891551,729.747234,753.342493
216,744.556624,741.899394,729.755001,753.350412
217,744.564238,741.907047,729.762615,753.358103
218,744.571777,741.914547,729.769963,753.365565


Based on what `tabel_ramal` points out, we see that the stock prices are predicted to always be higher than the yesterdays's price. We'll see the result clearer in a candlestick plot form. 

In [ ]:
df_date = pd.read_excel('/content/PJAA.JK Date Ramalan.xlsx')
date_final = list(df['Date'])
for i in list(df_date['Date']):
  date_final.append(i)
close_final = list(df['Close'])
for i in y_close.tolist():
  close_final.append(i)
open_final = list(df['Open'])
for i in y_open.tolist():
  open_final.append(i)
low_final = list(df['Low'])
for i in y_low.tolist():
  low_final.append(i)
high_final = list(df['High'])
for i in y_high.tolist():
  high_final.append(i)

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=date_final,
                open=open_final,
                high=high_final,
                low=low_final,
                close=close_final)])
fig.update_layout(
    title='Candlestick Plot Harga Saham PJAA Juli 2004 - September 2023',
    yaxis_title='Harga Saham',
    xaxis_title='Periode')
fig.show()

The candlestick plot above shows us that there is no significant trend in the increasing prices as per day the price will only scale up in a small value.

In [ ]:
high_low_check(tabel_ramal)

'100.0%'

After that, we check the forecasting result, whether it's already completing the satsifying condition based on 4 variables relation in this research or not. The code above turns the output that gives us the conclusion of no issues happened since it shows 100 in a correctness percentage.

This is just an addition, in case we need to specify the details of the best model architecture using `summary()`.

In [ ]:
model3.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 1, 6)              360       
                                                                 
 lstm_15 (LSTM)              (None, 5)                 240       
                                                                 
 dense_8 (Dense)             (None, 4)                 24        
                                                                 
Total params: 624
Trainable params: 624
Non-trainable params: 0
_________________________________________________________________


Building LSTM model is nearly same as building the statistical model for the inference. The friction between them is the LSTM model or a deep learning model we would say, does use the weights and biases that take a role of coefficients in a statistical model. In conclusion, statistical model uses coefficients for each variable, while a deep learning model uses weights and biases to predict the dependent variable value.

Below is the shape and values of weights and biases inside the Model 3. The details of the weights and biases are as follows.  
1. The first and second hidden layers contain 4 weights and biases for forget gate, input gate, cell state candidate gate, and output gate.
2. The third layer of the architecture is the output layer, which is Dense layer. So, it only turns the weight and bias matrices.

In [ ]:
bobot_layer1 = model3.layers[0].get_weights()
W1, U1, b1 = bobot_layer1
W1.shape, U1.shape, b1.shape

((8, 24), (6, 24), (24,))

In [ ]:
bobot_layer2 = model3.layers[1].get_weights()
W2, U2, b2 = bobot_layer2
W2.shape, U2.shape, b2.shape

((6, 20), (5, 20), (20,))

In [ ]:
bobot_layer3 = model6.layers[2].get_weights()
W3, U3, b3 = bobot_layer3
W3.shape, U3.shape, b3.shape

((6, 20), (5, 20), (20,))

In [ ]:
bobot_layer4 = model3.layers[2].get_weights()
W4, b4 = bobot_layer4
W4.shape, b4.shape

((5, 4), (4,))

In [ ]:
W1_i = W1[:, :6]
W1_f = W1[:, 6:6*2]
W1_c = W1[:, 6*6:7*3]
W1_o = W1[:, 6*3:]

U1_i = U1[:, :6]
U1_f = U1[:, 6:6*2]
U1_c = U1[:, 6*2:6*3]
U1_o = U1[:, 6*3:]

b1_i = b1[:6]
b1_f = b1[6:6*2]
b1_c = b1[6*2:6*3]
b1_o = b1[6*3:]

In [ ]:
W2_i = W2[:, :5]
W2_f = W2[:, 5:5*2]
W2_c = W2[:, 5*2:5*3]
W2_o = W2[:, 5*3:]

U2_i = U2[:, :5]
U2_f = U2[:, 5:5*2]
U2_c = U2[:, 5*2:5*3]
U2_o = U2[:, 5*3:]

b2_i = b2[:5]
b2_f = b2[5:5*2]
b2_c = b2[5*2:5*3]
b2_o = b2[5*3:]

In [ ]:
print(W1_i.shape)
print(W1_i)

In [ ]:
print(W1_f.shape)
print(W1_f)

In [ ]:
print(W1_c.shape)
print(W1_c)

In [ ]:
print(W1_o.shape)
print(W1_o)

In [ ]:
print(U1_i.shape)
print(U1_i)

In [ ]:
print(U1_f.shape)
print(U1_f)

In [ ]:
print(U1_c.shape)
print(U1_c)

In [ ]:
print(U1_o.shape)
print(U1_o)

In [ ]:
print(b1_i.shape)
print(b1_i)

In [ ]:
print(b1_f.shape)
print(b1_f)

In [ ]:
print(b1_c.shape)
print(b1_c)

In [ ]:
print(b1_o.shape)
print(b1_o)

In [ ]:
print(W2_i.shape)
print(W2_i)

In [ ]:
print(W2_f.shape)
print(W2_f)

In [ ]:
print(W2_c.shape)
print(W2_c)

In [ ]:
print(W2_o.shape)
print(W2_o)

In [ ]:
print(U2_i.shape)
print(U2_i)

In [ ]:
print(U2_f.shape)
print(U2_f)

In [ ]:
print(U2_c.shape)
print(U2_c)

In [ ]:
print(U2_o.shape)
print(U2_o)

In [ ]:
print(b2_i.shape)
print(b2_i)

In [ ]:
print(b2_f.shape)
print(b2_f)

In [ ]:
print(b2_c.shape)
print(b2_c)

In [ ]:
print(b2_o.shape)
print(b2_o)

In [ ]:
print(W4.shape)
print(W4)

(5, 4)
[[-1.5919706 -1.1290013 -1.1060009 -0.9588738]
 [ 1.3946352  1.5829324  1.8258307  1.5023379]
 [-0.813308  -1.2843193 -1.4863218 -0.8690771]
 [-1.7808238 -1.6436566 -1.4820477 -1.596047 ]
 [-1.2705185 -1.0980536 -0.6320374 -1.8055078]]


In [ ]:
print(b4.shape)
print(b4)

(4,)
[-0.14985594 -0.12470565 -0.20700966 -0.047617  ]
